In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import random
from keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Dense, LSTM, RepeatVector, TimeDistributed

Using TensorFlow backend.


### Definindo Paramêtros

In [2]:
data = 'train-mod.csv'
kval = 10
itertot = 40
sigma = 1.2
itergd = 300

### Calcula valor através da função escolhida

In [3]:
def transforminput(param, center):
    
    def rbfGaussiana(i, j, sigma):
        return np.exp(-(np.sum((param[i] - center[j])**2.0)**0.5) / sigma**2.0)
  
    def rbfMultiquadratica(i, j):
        return np.sqrt(np.sum(np.sum(param[i] - center[j])**2.0 + center[j]**2.0))/np.sum(center[j], axis=0)

    def rbfMultiquadraticaInversa(i, j):
        return np.sum(center[j], axis=0)/np.sqrt(np.sum(np.sum(param[i] - center[j])**2.0 + center[j]**2.0))
    
    newinput = np.zeros((len(param), len(center))).astype('float32')
    for i in range(len(param)):
        for j in range(len(center)):
            newinput[i,j] = rbfGaussiana(i, j, sigma)
    return newinput

In [4]:
def generatemodel(numparam):
    model = Sequential()
    model.add(Dense(1, input_dim=numparam, activation='sigmoid'))
    model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])
    return model

In [5]:
dataread = np.genfromtxt(data, delimiter=',')[1:,1:]

### Tratando entrada

In [6]:
alldata = []
for i in range(len(dataread)):
    if np.isnan(dataread[i,-2]):
        continue
    alldata.append(dataread[i])

alldata = np.asarray(alldata)

#dividing data

trainparam = alldata[:600,1:]
trainlabel = alldata[:600,0]
testparam = alldata[600:,1:]
testlabel = alldata[600:,0]

###############
#normalization#
###############

std = np.zeros((len(trainparam[0]))).astype('float32')
rata = np.zeros((len(trainparam[0]))).astype('float32')
trainparamnorm = np.zeros(np.shape(trainparam))
testparamnorm = np.zeros(np.shape(testparam))
for i in range(len(trainparam[0])):
    std[i] = np.std(trainparam[:,i])
    rata[i] = np.mean(trainparam[:,i])
    trainparamnorm[:,i] = (trainparam[:,i] - rata[i]) / std[i]
    testparamnorm[:,i] = (testparam[:,i] - rata[i]) / std[i]

### KMeans

In [7]:
kmean = np.zeros((kval, len(trainparamnorm[0])))
for i in range(kval):
    for j in range(len(kmean[0])):
        kmean[i,j] = random.uniform(min(trainparamnorm[:,j]),max(trainparamnorm[:,j]))
#looping of real algorithm
distmin = np.zeros((len(trainparamnorm)))
for i in range(itertot):
    print ('iteração para: ', i)
    for j in range(len(distmin)):
        #determine euclid distance
        distall = np.sum((trainparamnorm[j] - kmean)**2.0, axis=1)**0.5
        distmin[j] = np.argmin(distall)
#search new k mean
for j in range(kval):
    clust = []
    for k in range(len(distmin)):
        if distmin[k] == j:
            clust.append(trainparamnorm[k])
    if len(clust) > 0:
        kmean[j] = np.mean(np.asarray(clust), axis=0)

iteração para:  0
iteração para:  1
iteração para:  2
iteração para:  3
iteração para:  4
iteração para:  5
iteração para:  6
iteração para:  7
iteração para:  8
iteração para:  9
iteração para:  10
iteração para:  11
iteração para:  12
iteração para:  13
iteração para:  14
iteração para:  15
iteração para:  16
iteração para:  17
iteração para:  18
iteração para:  19
iteração para:  20
iteração para:  21
iteração para:  22
iteração para:  23
iteração para:  24
iteração para:  25
iteração para:  26
iteração para:  27
iteração para:  28
iteração para:  29
iteração para:  30
iteração para:  31
iteração para:  32
iteração para:  33
iteração para:  34
iteração para:  35
iteração para:  36
iteração para:  37
iteração para:  38
iteração para:  39


In [8]:
#tranform our input
newinput = transforminput(trainparamnorm, kmean)

print(trainlabel)

[0. 1. 1. 1. 0. 0. 0. 1. 1. 1. 1. 0. 0. 0. 1. 0. 0. 0. 1. 1. 1. 0. 1. 0.
 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 1. 0. 0. 0. 1. 1. 0. 1. 0. 1. 0. 0. 1. 0.
 0. 1. 0. 1. 0. 0. 0. 0. 0. 1. 0. 1. 1. 0. 1. 0. 1. 1. 0. 1. 0. 0. 0. 0.
 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 1. 0. 1. 1. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 1. 1. 0. 0. 0. 1. 0. 0.
 0. 0. 1. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.
 0. 0. 1. 1. 1. 0. 0. 1. 0. 1. 1. 1. 1. 0. 0. 0. 0. 0. 1. 0. 0. 1. 1. 1.
 0. 1. 0. 0. 1. 1. 0. 1. 0. 1. 0. 0. 1. 0. 1. 0. 0. 1. 0. 0. 1. 0. 0. 1.
 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 0. 0. 0. 0. 1.
 1. 1. 1. 1. 0. 1. 0. 0. 1. 0. 0. 0. 1. 0. 1. 0. 1. 1. 1. 1. 0. 0. 0. 0.
 0. 1. 0. 1. 1. 0. 1. 1. 1. 0. 0. 0. 1. 1. 0. 1. 1. 0. 0. 1. 1. 1. 0. 1.
 1. 1. 0. 0. 0. 0. 1. 1. 0. 1. 1. 0. 0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 1. 0.
 0. 0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 0. 1. 0. 0. 0. 1. 1. 0. 1. 0. 0. 1. 1.
 1. 1. 0. 1. 1. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0.

In [9]:
mod = generatemodel(kval)
mod.fit(newinput, trainlabel, batch_size=20, epochs=itergd, verbose=1, shuffle=True)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Epoch 1/300
600/600 [==============================] - 0s 215us/step - loss: 0.2564 - acc: 0.4217
Epoch 2/300
600/600 [==============================] - 0s 35us/step - loss: 0.2533 - acc: 0.4433
Epoch 3/300
600/600 [==============================] - 0s 37us/step - loss: 0.2504 - acc: 0.4967
Epoch 4/300
600/600 [==============================] - 0s 38us/step - loss: 0.2479 - acc: 0.5733
Epoch 5/300
600/600 [==============================] - 0s 37us/step - loss: 0.2458 - acc: 0.5933
Epoch 6/300
600/600 [==============================] - 0s 34us/step - loss: 0.2438 - acc: 0.5933
Epoch 7/300
600/600 [==============================] - 0s 46us/step - loss: 0.2421 - acc: 0.5933
Epoch 8/300
600/600 [==============================] - 0s 34us/step - loss: 0.2405 - acc: 0.5933
Epoch 9/300
600/600 [==============================] - 0s 36us/step - loss: 0.2392 - acc: 0.5933
Epoch 

600/600 [==============================] - 0s 33us/step - loss: 0.1935 - acc: 0.6883
Epoch 79/300
600/600 [==============================] - 0s 39us/step - loss: 0.1930 - acc: 0.6900
Epoch 80/300
600/600 [==============================] - 0s 36us/step - loss: 0.1926 - acc: 0.6900
Epoch 81/300
600/600 [==============================] - 0s 34us/step - loss: 0.1923 - acc: 0.6900
Epoch 82/300
600/600 [==============================] - 0s 35us/step - loss: 0.1919 - acc: 0.6900
Epoch 83/300
600/600 [==============================] - 0s 39us/step - loss: 0.1915 - acc: 0.6900
Epoch 84/300
600/600 [==============================] - 0s 39us/step - loss: 0.1911 - acc: 0.6883
Epoch 85/300
600/600 [==============================] - 0s 34us/step - loss: 0.1907 - acc: 0.6883
Epoch 86/300
600/600 [==============================] - 0s 34us/step - loss: 0.1904 - acc: 0.6883
Epoch 87/300
600/600 [==============================] - 0s 33us/step - loss: 0.1900 - acc: 0.6883
Epoch 88/300
600/600 [===========

600/600 [==============================] - 0s 34us/step - loss: 0.1730 - acc: 0.7600
Epoch 162/300
600/600 [==============================] - 0s 33us/step - loss: 0.1729 - acc: 0.7600
Epoch 163/300
600/600 [==============================] - 0s 33us/step - loss: 0.1728 - acc: 0.7600
Epoch 164/300
600/600 [==============================] - 0s 37us/step - loss: 0.1726 - acc: 0.7600
Epoch 165/300
600/600 [==============================] - 0s 35us/step - loss: 0.1724 - acc: 0.7600
Epoch 166/300
600/600 [==============================] - 0s 32us/step - loss: 0.1723 - acc: 0.7600
Epoch 167/300
600/600 [==============================] - 0s 32us/step - loss: 0.1722 - acc: 0.7600
Epoch 168/300
600/600 [==============================] - 0s 40us/step - loss: 0.1720 - acc: 0.7600
Epoch 169/300
600/600 [==============================] - 0s 33us/step - loss: 0.1719 - acc: 0.7600
Epoch 170/300
600/600 [==============================] - 0s 35us/step - loss: 0.1718 - acc: 0.7600
Epoch 171/300
600/600 [=

600/600 [==============================] - 0s 35us/step - loss: 0.1652 - acc: 0.7583
Epoch 244/300
600/600 [==============================] - 0s 32us/step - loss: 0.1651 - acc: 0.7583
Epoch 245/300
600/600 [==============================] - 0s 35us/step - loss: 0.1651 - acc: 0.7583
Epoch 246/300
600/600 [==============================] - 0s 36us/step - loss: 0.1650 - acc: 0.7583
Epoch 247/300
600/600 [==============================] - 0s 35us/step - loss: 0.1649 - acc: 0.7583
Epoch 248/300
600/600 [==============================] - 0s 32us/step - loss: 0.1649 - acc: 0.7583
Epoch 249/300
600/600 [==============================] - 0s 39us/step - loss: 0.1648 - acc: 0.7583
Epoch 250/300
600/600 [==============================] - 0s 41us/step - loss: 0.1647 - acc: 0.7567
Epoch 251/300
600/600 [==============================] - 0s 38us/step - loss: 0.1647 - acc: 0.7567
Epoch 252/300
600/600 [==============================] - 0s 39us/step - loss: 0.1646 - acc: 0.7567
Epoch 253/300
600/600 [=

In [10]:
newinputtest = transforminput(testparamnorm, kmean)

In [11]:
lifeprob = mod.predict(newinputtest)

### Armazenando os valores previstos

In [12]:
binpred = np.zeros((len(lifeprob)))
for i in range(len(lifeprob)):
    if lifeprob[i] > 0.5:
        binpred[i] = 1.

### Calculando a acurácia

In [13]:
score = 0
for i in range(len(testlabel)):
    if binpred[i] == testlabel[i]:
        score += 1
accbin = float(score) / float(len(testlabel))

In [14]:
errorabs = abs(score-len(lifeprob))
error = np.sum(errorabs/len(testlabel), axis=0)

In [15]:
brierscore = 0
for i in range(len(testlabel)):
    brierscore += (testlabel[i] - lifeprob[i])**2.0
brierscore = brierscore / float(len(testlabel))

In [16]:
for i in range(len(testlabel)):
    print (lifeprob[i], "-", testlabel[i])

[0.1466982] - 0.0
[0.6678958] - 1.0
[0.42319116] - 1.0
[0.1603204] - 0.0
[0.2387946] - 0.0
[0.17763874] - 0.0
[0.78699017] - 1.0
[0.1818001] - 0.0
[0.1496774] - 1.0
[0.8593124] - 1.0
[0.17217919] - 0.0
[0.78858685] - 1.0
[0.62282294] - 0.0
[0.17225349] - 0.0
[0.14508748] - 0.0
[0.15531951] - 0.0
[0.53610516] - 0.0
[0.5641421] - 1.0
[0.15676403] - 0.0
[0.5787039] - 1.0
[0.8083012] - 1.0
[0.5845121] - 1.0
[0.67020094] - 1.0
[0.43698764] - 0.0
[0.15261507] - 0.0
[0.15221056] - 0.0
[0.5980303] - 1.0
[0.25794056] - 0.0
[0.3658039] - 1.0
[0.5823026] - 0.0
[0.25224525] - 0.0
[0.15091997] - 0.0
[0.20971933] - 0.0
[0.6718865] - 1.0
[0.62221485] - 1.0
[0.16755912] - 0.0
[0.6325952] - 0.0
[0.18188712] - 0.0
[0.6438362] - 1.0
[0.58162516] - 1.0
[0.3801598] - 1.0
[0.15845272] - 1.0
[0.16995817] - 0.0
[0.4315831] - 0.0
[0.59812605] - 0.0
[0.20971933] - 0.0
[0.7185999] - 1.0
[0.15379542] - 0.0
[0.1813688] - 0.0
[0.18990228] - 0.0
[0.58070886] - 0.0
[0.1680955] - 0.0
[0.61616355] - 0.0
[0.17094007] - 

In [17]:
print ("Accuracy: ", accbin, "Brier score:", brierscore[0])
print ("Error: ", error)

Accuracy:  0.7982456140350878 Brier score: 0.14089157
Error:  0.20175438596491227
